<a href="https://colab.research.google.com/github/robertoarturomc/Data_Science/blob/master/Enlistar_Tablas_SAS_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
import os
import chardet

In [ ]:
# 21/05/24 Version 3, Roberto: Se adaptó el código al uso de funciones
#                                Arreglo de bugs menores

# 28/05/24 Version 4, Roberto: Se detecta el encoder automáticamente;
#                               Se ajustó para que cuando la definición de la macrovariable (%LET <macro>) no sucede en el presente código, el código funcione.
#                               Se agregaron patrones para hacer match con "PROC IMPORT FILE=(archivo)" y "PROC EXPORT FILE=(archivo)"
#                               Se cambiaron los patrones de OUTFILE= e INFILE para que fueran similares al de PROC IMPORT/EXPORT FILE.

In [ ]:
## Se leen los distintos archivos de la ruta especificada, y se pegan
def read_code(filename, encoding=None):
    code = ""
    if ( filename[-3:] == 'txt') or ( filename[-3:] == 'sas'):
        with open(filename, "rb") as my_file:
            print(filename)
            data = my_file.read()
            if encoding==None:
              encoding = chardet.detect(data)['encoding']
            code = code + data.decode(encoding)
            return code

def sas_code_cleaning(code):
    # Limpieza de string (código)
    code = code.upper()
    code = code.replace('\n', ' ')
    code = code.replace('\r', ' ')

    # Eliminar comentarios (código delimitado por "/*" y "*/")
    code_clean = re.sub(r'\/\*[\s\S]*?\*\/', "", code)

    return code_clean


In [ ]:
# Reemplazar macro variables (definidas como: LET XXX)
def replace_macro(code):

# Nota: hay usuarios que tienen la práctica de definir macro variables usando otras macrovariables.
# p.e. %LET BASE = F&PROCES_T._CUMAEST_CONTR_AJ;
# Por ese detalle, se hará el reemplazo uno por uno, de macro variables por su valor correspondiente

    read0 = """\%LET\s*(\w+)\s*\=\s*?([A-Z0-9/'"]+)"""

    code_nomacro = code
    while re.search(read0, code_nomacro) is not None:
        busq = re.search(read0, code_nomacro)

        match = busq.group(0)
        macro = busq.group(1)
        valor = busq.group(2)

        #Reemplazo la macrovariable por su valor
        code_nomacro = code_nomacro.replace('&' + macro + '.', valor)

        # Borro la definición de la macrovariable, para avanzar a la siguiente
        code_nomacro = code_nomacro.replace(match, '')

    # En el caso de que la definición de la macrovariable no esté en este código, cambio "&<variable>." por "<variable>"
    regex_macro_undefined = "(\&(\w+)\.)"

    macro_undefined = re.findall(regex_macro_undefined, code_nomacro)
    code_nomacro_undefined = code_nomacro
    for var in macro_undefined:
      code_nomacro_undefined = code_nomacro_undefined.replace(var[0], var[1])

    return code_nomacro_undefined


In [ ]:
## Patrones con regex para tablas que se leen y se escriben
def input_output_tables(code):
    read1 = "FROM\s*(?!WORK\.)(\w+)?\.(\w+)"
    read2 = "JOIN\s*(?!WORK\.)(\w+)?\.(\w+)"
    read3 = "SET\s*(?!WORK\.)(\w+)?\.(\w+)"
    read4 = 'INFILE\s*=\s*.*?(\w+)\.(\w+)'
    read5 = 'PROC\s*IMPORT\s*FILE\s*=\s*.*?(\w+)\.(\w+)'

    write1 = "CREATE TABLE\s*(?!WORK\.)(\w+)?\.(\w+)"
    write2 = "DATA\s*(?!WORK\.)(\w+)?\.(\w+)"
    write3 = 'OUTFILE\s*=\s*.*?(\w+)\.(\w+)'
    write4 = 'PROC\s*EXPORT\s*FILE\s*=\s*.*?(\w+)\.(\w+)'

    # Buscar matches dentro del código
    read_tabl = set(re.findall(read1, code) +
                    re.findall(read2, code) +
                    re.findall(read3, code) +
                    [(x[1], x[0]) for x in re.findall(read4, code)] +
                    [(x[1], x[0]) for x in re.findall(read5, code)] )

    write_tabl = set(re.findall(write1, code) +
                     re.findall(write2, code) +
                     [(x[1], x[0]) for x in re.findall(write3, code)] +
                     [(x[1], x[0]) for x in re.findall(write4, code)])

    # Determinación de Tablas Intermedias
    r_tables = read_tabl - write_tabl
    w_tables = write_tabl - read_tabl
    i_tables = read_tabl & write_tabl

    return r_tables, w_tables, i_tables

In [ ]:
# Se crea un DataFrame con todas la tablas
def df_tables(r_tables, w_tables, i_tables):
    r_tables =  pd.DataFrame(r_tables, columns=['Libref', 'Tabla'])
    r_tables["Tipo"] = "Input"

    i_tables =  pd.DataFrame(i_tables, columns=['Libref', 'Tabla'])
    i_tables["Tipo"] = "Intermedia"

    w_tables =  pd.DataFrame(w_tables, columns=['Libref', 'Tabla'])
    w_tables["Tipo"] = "Output"

    tables = pd.concat([r_tables, i_tables, w_tables])

    return tables

In [ ]:
## Matches con regex para info librerías
def info_libraries(code):
    read_libraries = """LIBNAME\s+(\w+)\s+([A-Z]*)\"?\'?\/?([A-Z0-9_/]*)?"""

    read_lib = re.findall(read_libraries, code)
    libraries = pd.DataFrame(read_lib, columns=['Base', 'Sistema', 'Path'])
    libraries = libraries.drop_duplicates()
    libraries.loc[libraries['Sistema'] == '', 'Sistema'] = 'SAS'
    return libraries


In [ ]:
# Juntar con Información de Bases
def merge_lib_tables(tables, libraries, proceso):
    full_tabl = tables.merge(libraries, left_on='Libref', right_on='Base', how='left')
    full_tabl["Proceso"] = proceso
    full_tabl[['Sistema']] = full_tabl[['Sistema']].fillna('Manual')

    full_tabl.drop(columns="Base", inplace=True)

    return full_tabl


In [ ]:
# Concentrado de Funciones para Tablas
def list_all(file):

    sas_code = read_code(file)
    code_clean = sas_code_cleaning(sas_code)
    code_nomacro = replace_macro(code_clean)

    r_tables, w_tables, i_tables = input_output_tables(code_nomacro)

    tables = df_tables(r_tables, w_tables, i_tables)
    libraries = info_libraries(code_clean)

    return tables, libraries

In [ ]:
# Se corren las funciones para cada uno de los archivos de la tabla:
full_tables = pd.DataFrame()
path = "/content/"


for filename in os.listdir(path):
  print(filename)
  if ( filename[-3:] == 'txt') or ( filename[-3:] == 'sas'):

    tables, libraries = list_all(path + filename)
    full_tables = pd.concat([full_tables, merge_lib_tables(tables, libraries, filename)], ignore_index=True)

full_tables

.config
4.-Ajustes BE2.sas
/content/4.-Ajustes BE2.sas
5.-Reserva Local IFRS9.sas
/content/5.-Reserva Local IFRS9.sas
7.-ajustes.sas
/content/7.-ajustes.sas
8.-Resumen base final2.sas
/content/8.-Resumen base final2.sas
2.-Asignaciones_base.sas
/content/2.-Asignaciones_base.sas
3.-Base CCI.sas
/content/3.-Base CCI.sas
6.-SEGM.sas
/content/6.-SEGM.sas
1.- librerias.sas
/content/1.- librerias.sas
sample_data


,Libref,Tabla,Tipo,Sistema,Path,Proceso
0,SERVIDOR,IFRS9_MES_BANCO,Output,Manual,NaN,5.-Reserva Local IFRS9.sas
1,KIDS6,VMIDSPDG,Input,Manual,NaN,3.-Base CCI.sas
2,KIDS6,VMIDSPLB,Input,Manual,NaN,3.-Base CCI.sas
3,SERVIDOR,IFRS9_MES_TOT7,Intermedia,SAS,FINANZAS4/CACC,3.-Base CCI.sas
4,SERVIDOR,IFRS9_TOTAL5,Intermedia,SAS,FINANZAS4/CACC,3.-Base CCI.sas
5,SERVIDOR,IFRS9_MES_TOT,Intermedia,SAS,FINANZAS4/CACC,3.-Base CCI.sas
6,SERVIDOR,IFRS9_MES_BANCO,Output,Manual,NaN,6.-SEGM.sas


In [ ]:
# Se leen los códigos y se unen:
full_tables = pd.DataFrame()
path = "/content/ArchivosFIN"

code=""

files= os.listdir(path)
files.sort()

for filename in files:
  #print(filename)
  if ( filename[-3:] == 'txt') or ( filename[-3:] == 'sas'):
    sas_code = read_code(filename)
    code = code + sas_code
    code_clean = sas_code_cleaning(code)
    code_nomacro = replace_macro(code_clean)

    r_tables, w_tables, i_tables = input_output_tables(code_nomacro)

    tables = df_tables(r_tables, w_tables, i_tables)
    libraries = info_libraries(code_clean)

    full_tables = pd.concat([full_tables, merge_lib_tables(tables, libraries, filename)], ignore_index=True)

full_tables

1.- librerias.sas
2.-Asignaciones_base.sas
3.-Base CCI.sas
4.-Ajustes BE2.sas
5.-Reserva Local IFRS9.sas
6.-SEGM.sas
7.-ajustes.sas
8.-Resumen base final2.sas


,Libref,Tabla,Tipo,Sistema,Path,Proceso
0,KIDS6,VMIDSPDG,Input,TERADATA,,3.-Base CCI.sas
1,KIDS6,VMIDSPLB,Input,TERADATA,,3.-Base CCI.sas
2,SERVIDOR,IFRS9_TOTAL5,Intermedia,SAS,FINANZAS4/CACC,3.-Base CCI.sas
3,SERVIDOR,IFRS9_ABR_TOT7,Intermedia,SAS,FINANZAS4/CACC,3.-Base CCI.sas
4,SERVIDOR,IFRS9_ABR_TOT,Intermedia,SAS,FINANZAS4/CACC,3.-Base CCI.sas
5,KIDS6,VMIDSPDG,Input,TERADATA,,4.-Ajustes BE2.sas
6,KIDS6,VMIDSPLB,Input,TERADATA,,4.-Ajustes BE2.sas
7,SERVIDOR,IFRS9_TOTAL5,Intermedia,SAS,FINANZAS4/CACC,4.-Ajustes BE2.sas
8,SERVIDOR,IFRS9_ABR_TOT7,Intermedia,SAS,FINANZAS4/CACC,4.-Ajustes BE2.sas
9,SERVIDOR,IFRS9_ABR_TOT,Intermedia,SAS,FINANZAS4/CACC,4.-Ajustes BE2.sas


In [ ]:
libraries

,Base,Sistema,Path
0,REG,SAS,FINANZAS6/MIS_REGULATORIOS
1,RAR_ABR,SAS,RAR3/SAS/GYS/APROVRAR/BD/ABR24
2,IFRS9ABR,SAS,UAIR_PIV/IFRS9/SALIDA/ABR24
3,SALABR,SAS,UAIR_PIV/IFRS9/MOTOR/SALIDA/ABR24/
4,DQ,SAS,FINANZAS/IPAB/IFRS9/DQ
5,SERVIDOR,SAS,FINANZAS4/CACC
6,SERVER1,SAS,FINANZAS4/IEBS/PROCESOS_IB/PRUEBAS
7,SLOCIFR,SAS,UAIR_PIV/IFRS9_IRB/MOTOR/SALIDA/FEB24
8,MOD,SAS,UAIR_PIV/IFRS9/MOTOR/MODULO_AJUSTES
9,CATAL,SAS,FINANZAS/IPAB/IFRS9/CATALOGOS


In [ ]:
full_tables.sort_values("Proceso", axis=0).reset_index(drop=True)

In [ ]:
full_tables.to_clipboard(index=False)

In [ ]:
ip = full_tables[(full_tables['Tipo']=="Input") | (full_tables['Tipo']=="Intermedia")]
op = full_tables[full_tables['Tipo']=="Output"]

ip.merge(op, on="Tabla", how="inner")